In [19]:
!nvidia-smi

Sun Nov 12 02:00:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 516.94       Driver Version: 516.94       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:02:00.0 Off |                  N/A |
| N/A   56C    P8    N/A /  N/A |   1558MiB /  2048MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Imports 

In [1]:
from keras.utils import np_utils
from keras.datasets import mnist
from keras.initializers import RandomNormal
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np
import time


### Ploting Functions

In [2]:
# Copied Code
def plt_dynamic(x, vy, ty, ax, color='b'):
    ax.plot(x, vy,'b', label='Validation Loss')
    ax.plot(x, ty, 'r', label='Train Loss')
    plt.legend()
    plt.grid()
    fig.canvas.draw()

In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

### Basic EDA

In [4]:
print(f"Train Example Count: {X_train.shape[0]}")
print(f"Test Example Count: {X_test.shape[0]}")
print(f"Image Size: {X_train.shape[1]} X {X_train.shape[2]}")

Train Example Count: 60000
Test Example Count: 10000
Image Size: 28 X 28


In [5]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1]*X_train.shape[2])

X_test = X_test.reshape(X_test.shape[0], X_test.shape[1]*X_test.shape[2])

In [6]:
X_train=X_train/255.
X_test=X_test/255.

In [7]:
y_train, y_test = np_utils.to_categorical(y_train, 10), np_utils.to_categorical(y_test, 10)

In [12]:
output_dim = 10
input_dim = X_train.shape[1]

batch_size = 2048
EPOCH = 20

In [13]:
def hyperparameter_best_fit(optim, loss_fnc, layer1Unit, layer2Unit, ):
    model = Sequential()
    model.add(Dense(layer1Unit, activation='relu', input_shape=(input_dim,)))
    model.add(Dense(layer2Unit, activation='relu'))
    model.add(Dense(output_dim, activation='softmax'))
    
    model.compile(optimizer=optim, loss=loss_fnc, metrics=['accuracy'])
    return model

In [14]:
optim = ['rmsprop', 'adam']
loss_fnc = ['KLDivergence', 'categorical_crossentropy']
layer1Unit = [512, 256]
layer2Unit = [128, 64]

In [15]:
model = KerasClassifier(build_fn=hyperparameter_best_fit, epochs=EPOCH, batch_size=batch_size, verbose=1)
param_grid = dict(optim=optim, loss_fnc=loss_fnc, layer1Unit=layer1Unit, layer2Unit=layer2Unit)

In [16]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=2)
grid_result = grid.fit(X_train, y_train)

Epoch 1/20
15/15 [==============================] - 2s 14ms/step - loss: 1.0647 - accuracy: 0.6695
Epoch 2/20
15/15 [==============================] - 0s 13ms/step - loss: 0.4225 - accuracy: 0.8796
Epoch 3/20
15/15 [==============================] - 0s 15ms/step - loss: 0.3242 - accuracy: 0.9050
Epoch 4/20
15/15 [==============================] - 0s 13ms/step - loss: 0.2885 - accuracy: 0.9140
Epoch 5/20
15/15 [==============================] - 0s 12ms/step - loss: 0.2362 - accuracy: 0.9288
Epoch 6/20
15/15 [==============================] - 0s 13ms/step - loss: 0.2178 - accuracy: 0.9345
Epoch 7/20
15/15 [==============================] - 0s 13ms/step - loss: 0.1695 - accuracy: 0.9507
Epoch 8/20
15/15 [==============================] - 0s 13ms/step - loss: 0.1628 - accuracy: 0.9501
Epoch 9/20
15/15 [==============================] - 0s 13ms/step - loss: 0.1450 - accuracy: 0.9566
Epoch 10/20
15/15 [==============================] - 0s 13ms/step - loss: 0.1261 - accuracy: 0.9628
Epoch 11/

In [18]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.970017 using {'layer1Unit': 512, 'layer2Unit': 128, 'loss_fnc': 'categorical_crossentropy', 'optim': 'adam'}
0.967367 (0.003133) with: {'layer1Unit': 512, 'layer2Unit': 128, 'loss_fnc': 'KLDivergence', 'optim': 'rmsprop'}
0.968683 (0.000117) with: {'layer1Unit': 512, 'layer2Unit': 128, 'loss_fnc': 'KLDivergence', 'optim': 'adam'}
0.970000 (0.001767) with: {'layer1Unit': 512, 'layer2Unit': 128, 'loss_fnc': 'categorical_crossentropy', 'optim': 'rmsprop'}
0.970017 (0.001117) with: {'layer1Unit': 512, 'layer2Unit': 128, 'loss_fnc': 'categorical_crossentropy', 'optim': 'adam'}
0.969967 (0.001600) with: {'layer1Unit': 512, 'layer2Unit': 64, 'loss_fnc': 'KLDivergence', 'optim': 'rmsprop'}
0.967800 (0.001000) with: {'layer1Unit': 512, 'layer2Unit': 64, 'loss_fnc': 'KLDivergence', 'optim': 'adam'}
0.968167 (0.000200) with: {'layer1Unit': 512, 'layer2Unit': 64, 'loss_fnc': 'categorical_crossentropy', 'optim': 'rmsprop'}
0.967367 (0.000133) with: {'layer1Unit': 512, 'layer2Unit': 64, 'los